# Assignment_9_DanieleSchiavo

### In the first part of the code we essesntially import all the modules, and the excel files where our data are, in order to assemble the data frame, containing the values of irradiance, temperture and a AC consumption.

### N.B
Once the data about temperature are taken from the excel file "Austin_weather_2014.csv", and gathered to the data frame with the irradiance and consumption data, we see that these value are lagged of 6 hours forward. The reason of this lag is due to the fact that the data about temperature are recorded with the Greenwich time zone. Therfore we should adapt these data to the Austin Time zone (6 hours backward with respect to the Greenwich one) but taking into account also that, the peak of tempearature is one hour after the irradiance one.

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
# importing the external files
ExternalFilesFolder =  r"C:\Users\elbar\Dropbox\python4ScientificComputing_Numpy_Pandas_MATPLotLIB\ExternalFiles"
ConsumptionFileName= "consumption_5545.csv"
TemperatureFileName= "Austin_weather_2014.csv"
IrradianceFileName= "irradiance_2014_gen.csv"

path_consumptionFile = os.path.join(ExternalFilesFolder,ConsumptionFileName)
path_TemperatureFile = os.path.join(ExternalFilesFolder,TemperatureFileName)
path_IrradianceFile = os.path.join(ExternalFilesFolder,IrradianceFileName)

DF_consumption = pd.read_csv(path_consumptionFile,sep=",", index_col=0)
DF_consumption.head()
DF_consumption.tail(10)

PreviousIndex = DF_consumption.index
NewParsedIndex= pd.to_datetime(PreviousIndex)
DF_consumption.index =NewParsedIndex 

DF_weather = pd.read_csv(path_TemperatureFile,sep=";",index_col=0)
previousIndex_weather=DF_weather.index
newIndex_weather=pd.to_datetime(previousIndex_weather)
DF_weather.index = newIndex_weather
DF_weather.columns
Series_Temperature = DF_weather["temperature"]

DF_Temperature= DF_weather[["temperature"]]


DF_irradianceSource = pd.read_csv(path_IrradianceFile,sep=";",index_col=1)

DF_irradiance=DF_irradianceSource[["gen"]]

DF_irradiance["gen"]<0
DF_irradiance[DF_irradiance["gen"]<0] = 0

DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])

DF_joined_cleaned = DF_joined.dropna()

DF_joined_cleaned_chosenDate=DF_joined_cleaned["2014-08-01":"2014-08-02"]
DF_joined_cleaned_chosenDate.head()

DF_joined_cleaned_chosenDate.describe()
DF_joined_cleaned_chosenDate["temperature"]=DF_joined_cleaned_chosenDate["temperature"].shift(-5)


DF_joined_cleaned_chosenDate_min=DF_joined_cleaned_chosenDate.min()
DF_joined_cleaned_chosenDate_max=DF_joined_cleaned_chosenDate.max()

DF_joined_cleaned_chosenDate_normalized=(DF_joined_cleaned_chosenDate-DF_joined_cleaned_chosenDate_min)/(DF_joined_cleaned_chosenDate_max-DF_joined_cleaned_chosenDate_min)

DF_joined_cleaned_chosenDate_normalized.plot()

C:\Users\elbar\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\elbar\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Introduction 

The main aim of this work is to manage and adjust all the data provided, creating a Data Frame with all the information. Infact, in order to predict the consumption "next hour" we need an huge amount of data whiich influence it.

First of all we need to create a copy of our data frame for thecnical reason. Infact python in order to minimize the memory used, modify the the value of a variable, every time it is changed, even in the previous steps.

In [16]:
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])
DF_mod=DF_joined.copy()

Then in order to make more intuitive the tabel, we change the name of the columns with a more pertinent attributes.

In [17]:
DF_mod=DF_mod.rename(columns={"air conditioner_5545":"AC_consumption","temperature":"temperature","gen":"irradiance"})


Plotting the value of temperature, irradiance and consumption, we can clearly see a lag among the peaks.
This is due to the thermal capacity of the wall during the winter.During the summer while, sun irradiance that came in through the windows, irradiance stuff in room and then, this irradiated stuff will consequently irradiates the ambient.But in our study case, we can see that the temperature peak is lagged for a longer time, this is due to an error in the data collection. Therefore the temperature value must be shifted five hours before. After the shifting, we will have "Not A Number" correspondence of the last five value of the temperature in the table, to delete this, we use the function .dropna().

In [18]:
DF_mod["temperature"]=DF_mod["temperature"].shift(-5)
DF_mod=DF_mod.dropna()

The consumption in the next hour, is strongly depend on the value of the temperature of the hours before. We should built a new columns where are rappresented the values of the temperatures of the 5 hours before. To do this we create a functon which take in input the Data Frame, the new column to add,the lag start and end and the interval's lag. We will see that this function is very useful also to create other columns with different type of lag. Into this function we can find a FOR cycle in which are fist created the columns name, then this new column is add in the data frame and filled whith the values of temperature shifted. Finaly we delate the "Not A Number" with dropna().

In [19]:
def lag_feature(df,column_name,lag_start,lag_end,lag_interval):
    for i in range(lag_start,lag_end,lag_interval):
        new_column_name= column_name + "-" + str(i) + "hr"
        df[new_column_name]=df[column_name].shift(i)
        df=df.dropna()
    return(df)

We apply this function to the temperature:

In [20]:
lag_start=1
lag_end=6
lag_interval=1
column_name= "temperature"
df=DF_mod

DF_mod=lag_feature(DF_mod,column_name,lag_start,lag_end,lag_interval)

C:\Users\elbar\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


We apply the function also to the irradiance and the consumption. In fact is common sense to think that the the consuption is dependant also to the value of irradiance and consumption in the hours before. Sperimentally we see that in order to predict the consumption we need the value of irradiance of the third up to sixth hours, and the value of the consumption of the hours up to the day before in order to understand the "Consumption behaviour" of the of the building.

In [21]:
DF_mod=lag_feature(DF_mod,"irradiance",3,6,1) 


C:\Users\elbar\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
DF_mod=lag_feature(DF_mod,"AC_consumption",1,24,1)


C:\Users\elbar\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


in order to know when the AC system is on or off, we sould add to our data frame the time. Infact if we think to an office, people will be in from 9:00 to 19:00 of Monday to Friday. We first add the hours:

In [12]:
DF_mod["hour"]=DF_mod.index.hour


Now we should impose a sort of continuity of the time, which is guarantee tanks the sine and cosine

In [13]:
DF_mod["sin_hour"]=np.sin(DF_mod.index.hour*np.pi/24)
DF_mod["cos_hour"]=np.cos(DF_mod.index.hour*np.pi/24)

Now we add other time parameters like:days of the week, months, and rhe weeks of the year:

In [14]:
DF_mod["DayOfWeek"]=DF_mod.index.dayofweek

DF_mod["Month"]=DF_mod.index.month

DF_mod["WeekOfYear"]=DF_mod.index.week

The last two columns will tell us if it is a work day, and if it is hour work. To do this we create two function to apply to our Data Frame. This function to detect the work day, takes as input the day of the week, and with an if cycle we put 1 if it is a Saturday or a Sunday (marked respectively with 5 and 6) and 0 otherwise. At the same way, the function of the detection of the work hours work: It takes as imput the hours of the day, if their value is between 9 and 19 the IF cycle willput 1, and 0 otherwise. Finaly we apply those function to our data frame

In [15]:
def weekendDetector(day):
    if (day==5 or day == 6):
        weeklabel=1
    else:
        weeklabel=0
    return(weeklabel)


def DayDetector(hour):
    if (hour<19 and hour >=9):
        daylabel=1
    else:
        daylabel=0
    return(daylabel)

DF_mod["Weekend"]=DF_mod["DayOfWeek"].apply(weekendDetector)
DF_mod["Working_Time"]=DF_mod["hour"].apply(DayDetector)

The final Data Frame is composed of 34 columns. The new Data Frame is now ready to be used in order to predict the consumption of the next hours.